 1	Hello, Data!


In [117]:
%pip install pandas
import pandas as pd

Note: you may need to restart the kernel to use updated packages.


In [118]:
primary_file_path = 'data/1000 Sales Records.csv'
secondary_file_path = 'data/worldcities.csv'

In [119]:

print("--- Primary Transaction Data (1000 Sales Records.csv) ---")
try:
    df_sales = pd.read_csv(primary_file_path, nrows=500)
    print("First 3 rows:")
    print(df_sales.head(3))  
    print("\nColumn names:")
    print(df_sales.columns)
    print(f"\nShape of loaded primary data: {df_sales.shape}")
except (FileNotFoundError, NotADirectoryError) as e:
    print(f"Error: {e}. Make sure '{primary_file_path}' is a valid file in the 'data' subfolder.")

print("\n" + "="*50 + "\n") 


--- Primary Transaction Data (1000 Sales Records.csv) ---
First 3 rows:
                         Region Country   Item Type Sales Channel  \
0  Middle East and North Africa   Libya   Cosmetics       Offline   
1                 North America  Canada  Vegetables        Online   
2  Middle East and North Africa   Libya   Baby Food       Offline   

  Order Priority  Order Date   Order ID   Ship Date  Units Sold  Unit Price  \
0              M  10/18/2014  686800706  10/31/2014        8446      437.20   
1              M   11/7/2011  185941302   12/8/2011        3018      154.06   
2              C  10/31/2016  246222341   12/9/2016        1517      255.28   

   Unit Cost  Total Revenue  Total Cost  Total Profit  
0     263.33     3692591.20  2224085.18    1468506.02  
1      90.93      464953.08   274426.74     190526.34  
2     159.42      387259.76   241840.14     145419.62  

Column names:
Index(['Region', 'Country', 'Item Type', 'Sales Channel', 'Order Priority',
       'Order Date'

In [120]:

print("--- Secondary Metadata (worldcities.csv) ---")
try:
    df_cities = pd.read_csv(secondary_file_path)
    print("First 3 rows:")
    print(df_cities.head(3))
    print("\nColumn names:")
    print(df_cities.columns)
    print(f"\nShape of loaded secondary data: {df_cities.shape}")
   
except FileNotFoundError:
    print(f"Error: The file {secondary_file_path} was not found. Make sure it's in the 'data' subfolder.")

--- Secondary Metadata (worldcities.csv) ---
First 3 rows:
          city   city_ascii      lat       lng        country iso2 iso3  \
0        Tokyo        Tokyo  35.6850  139.7514          Japan   JP  JPN   
1     New York     New York  40.6943  -73.9249  United States   US  USA   
2  Mexico City  Mexico City  19.4424  -99.1310         Mexico   MX  MEX   

         admin_name  capital  population          id  
0             Tōkyō  primary  35676000.0  1392685764  
1          New York      NaN  19354922.0  1840034016  
2  Ciudad de México  primary  19028000.0  1484247881  

Column names:
Index(['city', 'city_ascii', 'lat', 'lng', 'country', 'iso2', 'iso3',
       'admin_name', 'capital', 'population', 'id'],
      dtype='object')

Shape of loaded secondary data: (15493, 11)


2	Pick the Right Container

For storing individual transaction records, a class is the most suitable data structure in Python. This will allow one to encapsulate various data attributes in a transaction-say, date, product, price, etc.-and bundle behaviors related to that data, whether data cleaning or calculating totals via .clean() and .total() methods-so that the behaviors reside in the same place as the data they act upon. This leads to more organized, intuitive, and maintainable codebases.

3	Transaction Class and OO data structure	

In [121]:
import pandas as pd 


In [122]:
class Transaction:
    def __init__(self, date_val: str, customer_id_val: str, product_val: str,
                 price_val: Any, quantity_val: Any, coupon_code_val: Optional[str],
                 shipping_city_val: str, order_id_original_val: str,
                 region_val: Optional[str] = None, unit_cost_val: Optional[Any] = None,
                 sales_channel_val: Optional[str] = None, 
                 raw_total_revenue: Optional[Any] = None,
                 raw_total_cost: Optional[Any] = None,
                 raw_total_profit: Optional[Any] = None):

        self.date: Optional[datetime] = None
        try:
            self.date = pd.to_datetime(date_val).to_pydatetime()
        except Exception:
            self.date = None

        self.customer_id: str = str(customer_id_val)
        self.product: str = str(product_val)

        self.price: float = 0.0
        try:
            self.price = float(price_val)
        except (ValueError, TypeError):
            self.price = 0.0

        self.quantity: int = 0
        try:
            self.quantity = int(quantity_val)
        except (ValueError, TypeError):
            self.quantity = 0

        self.coupon_code: Optional[str] = coupon_code_val
        self.shipping_city: str = str(shipping_city_val)

        self.order_id_original: str = str(order_id_original_val)
        self.region_original: Optional[str] = str(region_val) if region_val else None
        self.sales_channel_original: Optional[str] = str(sales_channel_val) if sales_channel_val else None # Attribute to store it

        self.unit_cost_original: float = 0.0
        try:
            self.unit_cost_original = float(unit_cost_val) if unit_cost_val is not None else 0.0
        except (ValueError, TypeError):
            self.unit_cost_original = 0.0

        self.total_revenue_csv: float = 0.0
        try:
            self.total_revenue_csv = float(raw_total_revenue) if raw_total_revenue is not None else 0.0
        except (ValueError, TypeError):
            self.total_revenue_csv = 0.0

        self.discount_rate: float = 0.0
        self.discount_amount: float = 0.0
        self.subtotal: float = self.price * self.quantity
        self.final_total: float = self.subtotal
        self.days_since_purchase: Optional[int] = None

    def clean(self):
        self.product = self.product.strip().title()
        if self.price < 0:
            self.price = 0.0
        if self.quantity < 0:
            self.quantity = 0
        self.subtotal = self.price * self.quantity 
        self.final_total = self.subtotal * (1 - self.discount_rate) 

    def apply_discount(self, discount_rate: float):
        self.discount_rate = discount_rate
        self.subtotal = self.price * self.quantity 
        self.discount_amount = self.subtotal * self.discount_rate
        self.final_total = self.subtotal * (1 - self.discount_rate)

    def calculate_total(self):
        self.subtotal = self.price * self.quantity
        self.final_total = self.subtotal * (1 - self.discount_rate)
        return self.final_total

    def __repr__(self):
        date_str = self.date.strftime('%Y-%m-%d') if self.date else "N/A"
        return (f"Transaction(Date: {date_str}, CustID: {self.customer_id}, Prod: {self.product}, "
                f"Price: {self.price:.2f}, Qty: {self.quantity}, City: {self.shipping_city}, "
                f"Coupon: {self.coupon_code}, Final Total: {self.final_total:.2f})")

4	Bulk Loader

In [123]:

import random
from typing import List, Dict, Any, Optional 

def create_city_lookup_dict(cities_df: pd.DataFrame) -> Dict[str, str]:
    """
    Creates a lookup dictionary for country to its primary, admin, or first listed city.
    """
    if cities_df.empty or not all(col in cities_df.columns for col in ['country', 'city', 'population', 'capital']):
        return {}

    cities_df_sorted = cities_df.copy()

    cities_df_sorted['population'] = pd.to_numeric(cities_df_sorted['population'], errors='coerce').fillna(0)
    
    capital_priority = {'primary': 1, 'admin': 2}
    cities_df_sorted['capital_priority'] = cities_df_sorted['capital'].map(capital_priority).fillna(3)

    cities_df_sorted = cities_df_sorted.sort_values(
        by=['country', 'capital_priority', 'population'],
        ascending=[True, True, False] 
    )
    
    city_lookup = cities_df_sorted.groupby('country')['city'].first().to_dict()
    return city_lookup

def load_transactions(primary_filepath_arg: str, cities_df_arg: pd.DataFrame) -> List[Transaction]:
    """
    Loads transaction data from a CSV file path, enriches it with synthetic coupon codes
    and derived shipping cities, and returns a list of Transaction objects.
    """
    loaded_transactions_list: List[Transaction] = []
    
    try:
        sales_df = pd.read_csv(primary_filepath_arg, nrows=500)

        if sales_df.empty:
            print("Sales DataFrame is empty after loading.")
            return loaded_transactions_list

        coupon_options = [None, "SAVE10", "SAVE20", "FREESHIP15", "NOCODE"]
        weights = [0.5, 0.15, 0.15, 0.1, 0.1] 
        sales_df['coupon_code_synthetic'] = random.choices(coupon_options, weights=weights, k=len(sales_df))


        city_lookup = create_city_lookup_dict(cities_df_arg)
        if not city_lookup:
            print("City lookup dictionary is empty. Shipping cities may not be resolved correctly.")

       
        for _, row in sales_df.iterrows():
            transaction_country = str(row.get('Country', "Unknown Country")).strip()
            
            resolved_shipping_city = city_lookup.get(transaction_country, transaction_country)

           
            transaction_obj = Transaction(
                date_val=row['Order Date'],
                customer_id_val=str(row['Order ID']),               
                product_val=str(row['Item Type']),
                price_val=row['Unit Price'],
                quantity_val=row['Units Sold'],
                coupon_code_val=row['coupon_code_synthetic'],       
                shipping_city_val=resolved_shipping_city,           
                order_id_original_val=str(row['Order ID']),        
                region_val=str(row.get('Region')),               
                unit_cost_val=row.get('Unit Cost'),                 
                sales_channel_val=str(row.get('Sales Channel')),  
                raw_total_revenue=row.get('Total Revenue'),         
                raw_total_cost=row.get('Total Cost'),               
                raw_total_profit=row.get('Total Profit')            
            )
            loaded_transactions_list.append(transaction_obj)
            
    except FileNotFoundError:
        print(f"Error: The file {primary_filepath_arg} was not found.")
    except KeyError as e:
        print(f"KeyError: A required column is missing from the CSV - {e}. Check CSV headers and Transaction class arguments.")
    except Exception as e:
        print(f"An unexpected error occurred during transaction loading: {e}")
        
    return loaded_transactions_list


if 'df_cities' in globals() and 'primary_file_path' in globals():
    print("\n--- Loading All Transactions (First 500 from primary CSV) ---")
    all_transactions = load_transactions(primary_file_path, df_cities) 
    
    if all_transactions:
        print(f"Successfully loaded and created {len(all_transactions)} Transaction objects.")
        print("\nFirst 5 loaded Transaction objects:")
        for i in range(min(5, len(all_transactions))):
            print(all_transactions[i])
    else:
        print("No transactions were loaded. Check for errors above or if the primary CSV is empty/incorrectly formatted.")
else:
    print("\n'df_cities' (from worldcities.csv) or 'primary_file_path' (for sales data) is not defined.")
    print("Please ensure Step 1 (Hello, Data!) has been executed successfully.")


--- Loading All Transactions (First 500 from primary CSV) ---
Successfully loaded and created 500 Transaction objects.

First 5 loaded Transaction objects:
Transaction(Date: 2014-10-18, CustID: 686800706, Prod: Cosmetics, Price: 437.20, Qty: 8446, City: Tripoli, Coupon: FREESHIP15, Final Total: 3692591.20)
Transaction(Date: 2011-11-07, CustID: 185941302, Prod: Vegetables, Price: 154.06, Qty: 3018, City: Ottawa, Coupon: None, Final Total: 464953.08)
Transaction(Date: 2016-10-31, CustID: 246222341, Prod: Baby Food, Price: 255.28, Qty: 1517, City: Tripoli, Coupon: SAVE20, Final Total: 387259.76)
Transaction(Date: 2010-04-10, CustID: 161442649, Prod: Cereal, Price: 205.70, Qty: 3322, City: Tokyo, Coupon: FREESHIP15, Final Total: 683335.40)
Transaction(Date: 2011-08-16, CustID: 645713555, Prod: Fruits, Price: 9.33, Qty: 9845, City: N’Djamena, Coupon: NOCODE, Final Total: 91853.85)


5	Quick Profiling	

In [124]:
if 'all_transactions' in globals() and all_transactions:
    prices = [t.price for t in all_transactions if t.price is not None]
    shipping_cities = {t.shipping_city for t in all_transactions if t.shipping_city is not None}

    if prices:
        min_price = min(prices)
        max_price = max(prices)
        mean_price = sum(prices) / len(prices)
        print(f"--- Price Profiling ---")
        print(f"Minimum Price: {min_price:.2f}")
        print(f"Maximum Price: {max_price:.2f}")
        print(f"Mean Price: {mean_price:.2f}")
    else:
        print("No valid prices found for profiling.")

    print(f"\n--- Shipping City Profiling ---")
    print(f"Number of Unique Shipping Cities/Countries (Proxy): {len(shipping_cities)}")
else:
    print("Transaction list not loaded. Please run Step 4.")

--- Price Profiling ---
Minimum Price: 9.33
Maximum Price: 668.27
Mean Price: 274.30

--- Shipping City Profiling ---
Number of Unique Shipping Cities/Countries (Proxy): 171


6	Spot the Grime

In [125]:
if 'df_sales' in globals() and not df_sales.empty:
    print("--- Identifying Potential Data Grime ---")
    
    df_sales_grime_check = df_sales.copy()


    if len(df_sales_grime_check) > 2:
        df_sales_grime_check.loc[0, 'Item Type'] = '  cosmetics '  
        df_sales_grime_check.loc[1, 'Item Type'] = 'Vegetables  ' 
    print("Simulated: Whitespace/casing issues in 'Item Type' for first two records.")

    if len(df_sales_grime_check) > 3:
        df_sales_grime_check.loc[2, 'Unit Cost'] = np.nan
    print("Simulated: NaN in 'Unit Cost' for the third record.")


    if len(df_sales_grime_check) > 4:
        df_sales_grime_check.loc[3, 'Unit Price'] = -10.0
    print("Simulated: Negative 'Unit Price' for the fourth record.")


    print("\n--- Grime Check 1: 'Item Type' Formatting ---")
    original_item_types = df_sales_grime_check['Item Type'].unique()
    print(f"Unique 'Item Type' values (potential whitespace/casing issues): {original_item_types[:10]}") 
    

    changed_item_types_count = 0
    if 'Item Type' in df_sales_grime_check.columns:
        for item in df_sales_grime_check['Item Type'].dropna().astype(str): 
            if item != item.strip().title():
                changed_item_types_count += 1
    print(f"Number of 'Item Type' entries with leading/trailing whitespace or non-title case: {changed_item_types_count}")



    print("\n--- Grime Check 2: Missing 'Unit Cost' ---")
    if 'Unit Cost' in df_sales_grime_check.columns:
        missing_unit_costs = df_sales_grime_check['Unit Cost'].isnull().sum()
        print(f"Number of rows with missing 'Unit Cost': {missing_unit_costs}")
    else:
        print("'Unit Cost' column not found.")


    print("\n--- Grime Check 3: Negative 'Unit Price' ---")
    if 'Unit Price' in df_sales_grime_check.columns:
        negative_prices = df_sales_grime_check[df_sales_grime_check['Unit Price'] < 0]
        print(f"Number of rows with negative 'Unit Price': {len(negative_prices)}")
        if not negative_prices.empty:
            print("Example rows with negative 'Unit Price':")
            print(negative_prices[['Item Type', 'Unit Price']].head())
    else:
        print("'Unit Price' column not found.")
        
    
    print("\n--- Additional Original Grime Checks ---")

    if 'Order Priority' in df_sales_grime_check.columns:
        unique_priorities = df_sales_grime_check['Order Priority'].unique()
        print(f"4. Unique 'Order Priority' values found: {unique_priorities}")
    else:
        print("Column 'Order Priority' not found.")

    
    if all(col in df_sales_grime_check.columns for col in ['Units Sold', 'Unit Price', 'Total Revenue']):
        
        df_sales_grime_check['Temp Unit Price for Calc'] = df_sales_grime_check['Unit Price'].apply(lambda x: max(0, x))
        df_sales_grime_check['Calculated Revenue'] = df_sales_grime_check['Units Sold'] * df_sales_grime_check['Temp Unit Price for Calc']
        
        revenue_discrepancies = df_sales_grime_check[
            ~np.isclose(df_sales_grime_check['Total Revenue'], df_sales_grime_check['Calculated Revenue'])
        ]
        print(f"5. Number of rows where 'Total Revenue' != 'Units Sold' * 'Cleaned Unit Price': {len(revenue_discrepancies)}")
        if not revenue_discrepancies.empty:
            print("Example discrepancies (first 2):")
            print(revenue_discrepancies[['Units Sold', 'Unit Price', 'Total Revenue', 'Calculated Revenue']].head(2))
        df_sales_grime_check.drop(columns=['Temp Unit Price for Calc', 'Calculated Revenue'], inplace=True, errors='ignore')

    else:
        print("One or more financial columns for consistency check are missing.")

else:
    print("Sales DataFrame (df_sales) not loaded. Please run Step 1.")


--- Identifying Potential Data Grime ---
Simulated: Whitespace/casing issues in 'Item Type' for first two records.
Simulated: NaN in 'Unit Cost' for the third record.
Simulated: Negative 'Unit Price' for the fourth record.

--- Grime Check 1: 'Item Type' Formatting ---
Unique 'Item Type' values (potential whitespace/casing issues): ['  cosmetics ' 'Vegetables  ' 'Baby Food' 'Cereal' 'Fruits' 'Clothes'
 'Vegetables' 'Snacks' 'Household' 'Cosmetics']
Number of 'Item Type' entries with leading/trailing whitespace or non-title case: 2

--- Grime Check 2: Missing 'Unit Cost' ---
Number of rows with missing 'Unit Cost': 1

--- Grime Check 3: Negative 'Unit Price' ---
Number of rows with negative 'Unit Price': 1
Example rows with negative 'Unit Price':
  Item Type  Unit Price
3    Cereal       -10.0

--- Additional Original Grime Checks ---
4. Unique 'Order Priority' values found: ['M' 'C' 'H' 'L']
5. Number of rows where 'Total Revenue' != 'Units Sold' * 'Cleaned Unit Price': 1
Example discr

7	Cleaning Rules

In [126]:
if 'all_transactions' in globals() and all_transactions:
    print("--- Applying Cleaning Rules to all Transaction objects ---")
    
    
    
    if len(all_transactions) > 0:
    
        print(f"\nExample 1: Product Name Cleaning")
       
        all_transactions[0].product = "  fruits  " 
        print(f"  Transaction 0 Product (Before clean): '{all_transactions[0].product}'")
        all_transactions[0].clean() 
        print(f"  Transaction 0 Product (After clean): '{all_transactions[0].product}'")

        
        if len(all_transactions) > 1:
            print(f"\nExample 2: Negative Price Cleaning")
            all_transactions[1].price = -50.0 
            all_transactions[1].quantity = 2  
            all_transactions[1].calculate_total() 
            print(f"  Transaction 1 (Before clean): Price={all_transactions[1].price}, FinalTotal={all_transactions[1].final_total}")
            all_transactions[1].clean() 
            print(f"  Transaction 1 (After clean): Price={all_transactions[1].price}, FinalTotal={all_transactions[1].final_total}")
    
   
    print("\nApplying .clean() to all remaining transactions...")
    for t_idx in range(len(all_transactions)): 
        if t_idx > 1: 
             all_transactions[t_idx].clean()
    print("Cleaning applied to all transactions.")

   
    negative_prices_after_all_cleaned = sum(1 for t in all_transactions if t.price < 0)
    print(f"\nNumber of transactions with negative prices after all have been cleaned: {negative_prices_after_all_cleaned}")

   
    print("\nFirst 3 product names after all transactions cleaned:")
    for i in range(min(3, len(all_transactions))):
        print(f"  Transaction {i} Product: '{all_transactions[i].product}'")
else:
    print("Transaction list (all_transactions) not loaded or empty. Please run Step 4.")


--- Applying Cleaning Rules to all Transaction objects ---

Example 1: Product Name Cleaning
  Transaction 0 Product (Before clean): '  fruits  '
  Transaction 0 Product (After clean): 'Fruits'

Example 2: Negative Price Cleaning
  Transaction 1 (Before clean): Price=-50.0, FinalTotal=-100.0
  Transaction 1 (After clean): Price=0.0, FinalTotal=0.0

Applying .clean() to all remaining transactions...
Cleaning applied to all transactions.

Number of transactions with negative prices after all have been cleaned: 0

First 3 product names after all transactions cleaned:
  Transaction 0 Product: 'Fruits'
  Transaction 1 Product: 'Vegetables'
  Transaction 2 Product: 'Baby Food'


8	Transformations

In [127]:
def get_discount_rate_from_coupon(coupon_code: Optional[str]) -> float:
    """Maps a coupon code string to a numeric discount rate."""
    if coupon_code == "SAVE10":
        return 0.10
    elif coupon_code == "SAVE20":
        return 0.20
    elif coupon_code == "SUMMER25":
        return 0.25
    elif coupon_code == "FREESHIP": 
        return 0.15 
    else: 
        return 0.0

if 'all_transactions' in globals() and all_transactions:
    print("--- Applying Coupon Code Transformations & Updating Totals for ALL Transactions ---")
    
    sum_final_total_before_coupon_logic = 0
    for t_before in all_transactions:
        t_before.calculate_total() 
        sum_final_total_before_coupon_logic += t_before.final_total

    print("\nSample of first 3 transactions (state before this systematic discount application):")
    for i in range(min(3, len(all_transactions))):
        t = all_transactions[i]
        print(f"  ID: {t.order_id_original}, Coupon: {t.coupon_code}, Subtotal: {t.subtotal:.2f}, Current Discount Rate: {t.discount_rate:.2f}, Final Total: {t.final_total:.2f}")

    
    transactions_with_discount_applied = 0
    for t in all_transactions:
        new_discount_rate = get_discount_rate_from_coupon(t.coupon_code)
        t.apply_discount(new_discount_rate) 
        if new_discount_rate > 0:
            transactions_with_discount_applied += 1
    
    sum_final_total_after_coupon_logic = sum(t.final_total for t in all_transactions)

    print("\nSample of first 3 transactions (state after this systematic discount application):")
    for i in range(min(3, len(all_transactions))):
        t = all_transactions[i] 
        print(f"  ID: {t.order_id_original}, Coupon: {t.coupon_code}, Subtotal: {t.subtotal:.2f}, New Discount Rate: {t.discount_rate:.2f}, Discount Amount: {t.discount_amount:.2f}, Final Total: {t.final_total:.2f}")

    print("\n--- Transformation Summary for All Transactions ---")
    print(f"Total transactions processed: {len(all_transactions)}")
    print(f"Number of transactions that received a discount: {transactions_with_discount_applied}")
    print(f"Sum of all final_totals BEFORE coupon transformation: {sum_final_total_before_coupon_logic:.2f}")
    print(f"Sum of all final_totals AFTER coupon transformation: {sum_final_total_after_coupon_logic:.2f}")
    
else:
    print("Transaction list (all_transactions) not loaded or empty. Please run Step 4 & 7.")

--- Applying Coupon Code Transformations & Updating Totals for ALL Transactions ---

Sample of first 3 transactions (state before this systematic discount application):
  ID: 686800706, Coupon: FREESHIP15, Subtotal: 3692591.20, Current Discount Rate: 0.00, Final Total: 3692591.20
  ID: 185941302, Coupon: None, Subtotal: 0.00, Current Discount Rate: 0.00, Final Total: 0.00
  ID: 246222341, Coupon: SAVE20, Subtotal: 387259.76, Current Discount Rate: 0.00, Final Total: 387259.76

Sample of first 3 transactions (state after this systematic discount application):
  ID: 686800706, Coupon: FREESHIP15, Subtotal: 3692591.20, New Discount Rate: 0.00, Discount Amount: 0.00, Final Total: 3692591.20
  ID: 185941302, Coupon: None, Subtotal: 0.00, New Discount Rate: 0.00, Discount Amount: 0.00, Final Total: 0.00
  ID: 246222341, Coupon: SAVE20, Subtotal: 387259.76, New Discount Rate: 0.20, Discount Amount: 77451.95, Final Total: 309807.81

--- Transformation Summary for All Transactions ---
Total tra

9	Feature Engineering

In [128]:
if 'all_transactions' in globals() and all_transactions:
    print("--- Feature Engineering: Adding 'days_since_purchase' ---")
    
    valid_dates = [t.date for t in all_transactions if t.date is not None]
    if not valid_dates:
        print("No valid dates in transactions to determine a reference date. Skipping 'days_since_purchase'.")
        for t in all_transactions:
            t.days_since_purchase = None
    else:
        current_processing_date = max(valid_dates) 
        print(f"Reference date for 'days_since_purchase' calculation: {current_processing_date.strftime('%Y-%m-%d')}")

        for t in all_transactions:
            if t.date: 
                t.days_since_purchase = (current_processing_date - t.date).days
            else:
                t.days_since_purchase = None 
            
            t.calculate_total() 

        print("\nFirst 3 transactions with engineered 'days_since_purchase' and financial details:")
        for i in range(min(3, len(all_transactions))):
            t = all_transactions[i]
            date_str = t.date.strftime('%Y-%m-%d') if t.date else 'N/A'
            print(f"  Transaction OrderID: {t.order_id_original}, Date: {date_str}, "
                  f"Subtotal: {t.subtotal:.2f}, DiscountAmt: {t.discount_amount:.2f}, FinalTotal: {t.final_total:.2f}, "
                  f"DaysSincePurchase: {t.days_since_purchase}")
else:
    print("Transaction list (all_transactions) not loaded or empty. Please run previous steps.")

--- Feature Engineering: Adding 'days_since_purchase' ---
Reference date for 'days_since_purchase' calculation: 2017-07-26

First 3 transactions with engineered 'days_since_purchase' and financial details:
  Transaction OrderID: 686800706, Date: 2014-10-18, Subtotal: 3692591.20, DiscountAmt: 0.00, FinalTotal: 3692591.20, DaysSincePurchase: 1012
  Transaction OrderID: 185941302, Date: 2011-11-07, Subtotal: 0.00, DiscountAmt: 0.00, FinalTotal: 0.00, DaysSincePurchase: 2088
  Transaction OrderID: 246222341, Date: 2016-10-31, Subtotal: 387259.76, DiscountAmt: 77451.95, FinalTotal: 309807.81, DaysSincePurchase: 268


10	Mini-Aggregation

In [129]:
if 'all_transactions' in globals() and all_transactions:
    revenue_per_city: Dict[str, float] = {}
    for t in all_transactions:
        current_final_total = t.final_total if isinstance(t.final_total, (int, float)) and pd.notna(t.final_total) else 0.0
        
        city_key = t.shipping_city if pd.notna(t.shipping_city) else "Unknown City"
        revenue_per_city[city_key] = revenue_per_city.get(city_key, 0.0) + current_final_total

    print("--- Revenue Per Shipping City/Country (Proxy) ---")
    if revenue_per_city:
        sorted_revenue_per_city = sorted(revenue_per_city.items(), key=lambda item: item[1], reverse=True)
        
        print(f"{'City/Country':<30} | {'Total Revenue':>15}")
        print("-" * 48)
        for city, total_revenue in sorted_revenue_per_city:
            print(f"{city:<30} | {total_revenue:>15.2f}")
    else:
        print("Could not calculate revenue per city. Aggregation dictionary is empty.")
else:
    print("Transaction list (all_transactions) not loaded or empty. Please run previous steps.")

--- Revenue Per Shipping City/Country (Proxy) ---
City/Country                   |   Total Revenue
------------------------------------------------
Port Moresby                   |     15629197.78
San José                       |     15578320.59
Lisbon                         |     12486365.66
Czech Republic                 |     11925961.58
Havana                         |     11889494.51
United States of America       |     11837837.23
Nuku‘alofa                     |     11821702.00
Cape Town                      |     11276161.63
Tbilisi                        |     11210683.14
N’Djamena                      |     11141002.35
Luxembourg                     |     10875934.86
Vienna                         |      9889550.02
Mbabane                        |      9840650.51
Ankara                         |      9787768.05
Hanoi                          |      9437912.38
Maputo                         |      8721343.91
Sisimiut                       |      8551967.53
Bangui             


11	Serialization Checkpoint

In [130]:
import json 

if 'all_transactions' in globals() and all_transactions:
    print("--- Serializing Cleaned and Enriched Transaction Data ---")
    transactions_dict_list = []
    for t in all_transactions:
        transactions_dict_list.append({
            'date': t.date.strftime('%Y-%m-%d %H:%M:%S') if t.date else None,
            'customer_id': t.customer_id,
            'product': t.product,
            'price': t.price,
            'quantity': t.quantity,
            'coupon_code': t.coupon_code,
            'shipping_city': t.shipping_city,
            'order_id_original': t.order_id_original,
            'region_original': t.region_original,
            'sales_channel_original': t.sales_channel_original,
            'unit_cost_original': t.unit_cost_original,
            'total_revenue_csv': t.total_revenue_csv, 
            'discount_rate': t.discount_rate,
            'discount_amount': t.discount_amount,
            'subtotal': t.subtotal,                 
            'final_total': t.final_total,           
            'days_since_purchase': t.days_since_purchase
        })

    df_cleaned_transactions = pd.DataFrame(transactions_dict_list)

    json_output_path = 'data/cleaned_transactions.json'
    parquet_output_path = 'data/cleaned_transactions.parquet'
    csv_output_path = 'data/cleaned_transactions.csv'

    try:
        df_cleaned_transactions.to_json(json_output_path, orient='records', lines=True, date_format='iso', default_handler=str)
        print(f"Successfully saved cleaned data to JSON: {json_output_path}")

        df_cleaned_transactions.to_csv(csv_output_path, index=False)
        print(f"Successfully saved cleaned data to CSV: {csv_output_path}")
        
        try:
            df_cleaned_transactions.to_parquet(parquet_output_path, index=False, engine='pyarrow') 
            print(f"Successfully saved cleaned data to Parquet: {parquet_output_path}")
        except ImportError:
            print("\nError: 'pyarrow' library is required to save to Parquet. Parquet file not saved.")
            print("Please install it: pip install pyarrow (or conda install pyarrow if using Conda)")
        except Exception as e_parquet:
            print(f"\nAn error occurred during Parquet serialization: {e_parquet}. Parquet file may not be saved correctly.")

    except Exception as e:
        print(f"\nAn error occurred during general serialization: {e}")
else:
    print("Transaction list (all_transactions) not loaded or is empty. Please run previous steps to generate data.")

--- Serializing Cleaned and Enriched Transaction Data ---
Successfully saved cleaned data to JSON: data/cleaned_transactions.json
Successfully saved cleaned data to CSV: data/cleaned_transactions.csv
Successfully saved cleaned data to Parquet: data/cleaned_transactions.parquet


12	Soft Interview Reflection

Object-Oriented Programming (OOP) found full expression here, employing the Transaction class to bundle transaction data (price, product) with its manufactures or operations (clean(), apply_discount(), calculate_final_total()). By keeping the data and methods organized and logically grouped, cleaning, transformation, and feature engineering could all be neatly implemented with shared code applied equally to every record. For instance, discounting logic or cleaning rules that were complex would reside within those class methods, becoming easier to read and maintain. Such a structured way of working is a great way of managing complex data pipelines and developing in teams, much of the real-world engineering data are legible and reusable.

In [ ]:
## Data Dictionary


| Field Name                    | Type (Python/Pandas)         | Description                                                                 | Source                           |
|-------------------------------|------------------------------|-----------------------------------------------------------------------------|-----------------------------------|
| **Order Date (original)**     | object / string              | The original date string when the order was placed.                         | Primary CSV                      |
| **Order ID (original)**       | int64 / string               | Unique identifier for the order from the source CSV.                        | Primary CSV                      |
| **Item Type (original)**      | object / string              | The original type or category of the item sold.                             | Primary CSV                      |
| **Units Sold (original)**     | int64                        | The number of units of the item sold in the transaction, from source.       | Primary CSV                      |
| **Unit Price (original)**     | float64                      | The price of a single unit of the item, from source.                        | Primary CSV                      |
| **Unit Cost (original CSV)**  | float64                      | The cost of a single unit of the item to the seller, from source.           | Primary CSV                      |
| **Total Revenue (original CSV)** | float64                   | The total revenue recorded in the original CSV for the transaction.          | Primary CSV                      |
| **Sales Channel (original)**  | object / string              | The channel through which the sale was made (e.g., Online, Offline).        | Primary CSV                      |
| **Country (original)**        | object / string              | The country associated with the transaction from source.                    | Primary CSV                      |
| **Region (original CSV)**     | object / string              | The region associated with the transaction from source.                     | Primary CSV                      |
| **city (worldcities)**        | string / object              | Name of the city from the secondary dataset.                                | worldcities.csv                  |
| **country (worldcities)**     | string / object              | Name of the country the city belongs to, from the secondary dataset.        | worldcities.csv                  |
| **population (worldcities)**  | float64                      | Population of the city (used if selecting most populous).                   | worldcities.csv                  |
| **capital (worldcities)**     | string / object              | Type of capital (e.g. primary, admin) for the city.                         | worldcities.csv                  |
| **date**                      | datetime                     | Processed order date as a Python datetime object.                           | Engineered from 'Order Date'     |
| **customer_id**               | string                       | Customer identifier, proxied by 'Order ID'.                                 | Engineered from 'Order ID'       |
| **product**                   | string                       | Cleaned and standardized item type (e.g., title cased, stripped whitespace).| Engineered from 'Item Type'      |
| **price**                     | float                        | Cleaned unit price as a float (e.g., non-negative).                         | Engineered from 'Unit Price'     |
| **quantity**                  | int                          | Cleaned units sold as an integer (e.g., non-negative).                      | Engineered from 'Units Sold'     |
| **coupon_code**               | string / None                | Synthetically generated coupon code applied to the transaction.             | Engineered (Synthetic)           |
| **shipping_city**             | string                       | Derived shipping city, based on 'Country' (from sales) and worldcities.csv. | Engineered                       |
| **order_id_original**         | string                       | Stored original Order ID from the sales data.                               | From Primary CSV via Transaction |
| **region_original**           | string / None                | Stored original Region from the sales data.                                 | From Primary CSV via Transaction |
| **sales_channel_original**    | string / None                | Stored original Sales Channel from the sales data.                          | From Primary CSV via Transaction |
| **unit_cost_original**        | float                        | Stored original Unit Cost from the sales data (defaulted to 0.0 if missing/invalid). | From Primary CSV via Transaction |
| **total_revenue_csv**         | float                        | Stored original Total Revenue from the sales data (defaulted to 0.0 if missing/invalid). | From Primary CSV via Transaction |
| **discount_rate**             | float                        | Numeric discount rate (0.0 to 1.0) derived from coupon_code.                | Engineered from coupon_code      |
| **discount_amount**           | float                        | Calculated monetary value of the discount (subtotal * discount_rate).       | Engineered                       |
| **subtotal**                  | float                        | Calculated price * quantity (using cleaned price/quantity) before discount. | Engineered                       |
| **final_total**               | float                        | Calculated final revenue after applying discounts (subtotal - discount_amount). | Engineered                   |
| **days_since_purchase**       | int / None                   | Number of days from the transaction date to the latest date in the dataset.  | Engineered from date             |

References:
1. https://www.kaggle.com/datasets/viswanathanc/world-cities-datasets
2. https://excelbianalytics.com/wp/downloads-18-sample-csv-files-data-sets-for-testing-sales/